### HANA Cloud 연결

In [1]:
import pandas as pd
import json
import os
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

2.2.2
2.21.24072600


In [2]:
with open(os.path.join(os.getcwd(), '../config/cesco-poc-hc-service-key.json')) as f:
    hana_env_c = json.load(f)
    port_c = hana_env_c['port']
    user_c = hana_env_c['user']
    host_c = hana_env_c['host']
    pwd_c = hana_env_c['pwd']

cc = ConnectionContext(address=host_c, port=port_c, user=user_c, password=pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check hana_ml version
print(cc.hana_version())
#Check current schema
print(cc.get_current_schema())

4.00.000.00.1721126161 (fa/CE2024.14)
GEN_AI


In [ ]:
#FileNames Sequence
sql_command = '''CREATE SEQUENCE GEN_AI.PROBLEM_NO START WITH 1 INCREMENT BY 1'''
cursor.execute(sql_command)
cursor.close()

### 📃CESCO_ProblemSolutions

In [3]:
#CESCO_ProblemSolutions 생성
#테이블 한글명: 문의 DB
#테이블 설명: 기존 핼프데스크 OA 질의 응답 저장 및 문제 벡터값 저장
#작성일: 2024-08-07

cursor = cc.connection.cursor()

sql_command = '''CREATE TABLE "CESCO_PROBLEMSOLUTIONS"(
    "ProblemID" BIGINT PRIMARY KEY,
    "ProblemDescription" NVARCHAR(1000),
    "ProblemCategory" NVARCHAR(100),
    "ProblemKeyword" NVARCHAR(100),
    "Solution" NCLOB MEMORY THRESHOLD 0,
    "SolutionDoc" NCLOB MEMORY THRESHOLD 0,
    "AdditionalInfo" NCLOB MEMORY THRESHOLD 0,
    "Code" NVARCHAR(500),
    "VectorProblem" REAL_VECTOR(1536),
    "CreateDate" DATE,
    "UpdateDate" DATE
)
'''

cursor.execute(sql_command)
cursor.close()


### 📃CESCO_UnansweredQuestions

In [55]:
#CESCO_UnansweredQuestions 생성
#테이블 한글명: 미응답질문DB
#테이블 설명: CESCO BOT이 답변하지 못한 사용자 질문을 담아 신규 질문 추가 여부 및 질문 분석 등에 활용한다.
#작성일: 2024-08-07

cursor = cc.connection.cursor()

sql_command = '''CREATE TABLE "CESCO_UNANSWEREDQUESTIONS"(
    "QuestionID" BIGINT PRIMARY KEY,
    "QuestionText" NVARCHAR(1000),
    "Status" NVARCHAR(100),
    "StatusUpdateDate" DATE,
    "DownloadDate" DATE,
    "CreateDate" DATE
)
'''

cursor.execute(sql_command)
cursor.close()


In [60]:
#FileNames Sequence
sql_command = '''CREATE SEQUENCE GEN_AI.FILE_NO START WITH 1 INCREMENT BY 1'''
cursor.execute(sql_command)
cursor.close()

### 📃CESCO_FileNames

In [4]:
#CESCO_ProblemSolutions 생성
#테이블 한글명: 미응답질문DB
#테이블 설명: CESCO BOT이 답변하지 못한 사용자 질문을 담아 신규 질문 추가 여부 및 질문 분석 등에 활용한다.
#(+) 원본 테이블 관리 페이지에서 활용한다
#작성일: 2024-08-07
#수정일: 2024-08-27

cursor = cc.connection.cursor()

sql_command = '''CREATE TABLE "CESCO_FILENAMES"(
    "CodeID" BIGINT GENERATED BY DEFAULT AS IDENTITY,
    "Code" NVARCHAR(500),
    "FileName" NVARCHAR(100),
    "CreateDate" DATE,
    PRIMARY KEY ("CodeID")
)
'''

cursor.execute(sql_command)
cursor.close()

### 📃CESCO_UNANSWEREDQUESTIONS

미응답 가데이터 TEST용

In [4]:
test_data = [
    "SC 법인 차량에 에어컨이 안나옵니다.",
    "차세대시스템 내 DB 구조가 궁금합니다.",
    "DATA를 DB에 올리는 방법이 궁금합니다.",
    "바퀴벌레 해충 동정 법이 궁금합니다.",
    "시궁쥐와 들쥐의 차이점이 궁금합니다.",
    "세스코 터치센터의 주소가 궁금해요.",
    "세스카페의 아메리카노 가격이 궁금합니다.",
    "세스코 터치센터 근처 버스정류장이 어디있나요",
    "터치센터에서 강일역 가능 방법이 궁금해요",
    "셔틀버스 탑승 시간을 알려주세요.",
    "출근 할 때 잠실역에서 셔틀버스 어디서 타는지 알고 싶어요.",
    "본사 직원 출근 시간이 궁금합니다.",
    "IT 서비스 데스크 직통 번호를 알려주세요.",
    "회사에서 지급한 노트북이 고장났어요.",
    "윈도우 설치 방법이 궁금합니다.",
    "세스톡 로그인이 안됩니다.",
    "연차 신청 방법이 궁금합니다.",
    "내 연차가 얼마나 남았는지 궁금해.",
    "이번 달 월급 명세서 확인하려면 어디로 가야하나요",
    "회사창립기념일이 언제인가요?"
]

In [6]:
#FileNames Sequence
sql_command = '''CREATE SEQUENCE GEN_AI.UNANSWER_NO START WITH 1 INCREMENT BY 1'''
cursor.execute(sql_command)
cursor.close()

In [5]:
cursor = cc.connection.cursor()

for text in test_data:
    sql_command = '''INSERT INTO "CESCO_UNANSWEREDQUESTIONS" (
    "QuestionID", "QuestionText", "Status", "StatusUpdateDate", "DownloadDate", "CreateDate")
    VALUES (GEN_AI.UNANSWER_NO.NEXTVAL, '{text}', '미처리', CURRENT_DATE, CURRENT_DATE, CURRENT_DATE)'''.format(text=text)

    try:
        cursor.execute(sql_command)
    except Exception as e:
        cc.connection.rollback()
        print("An error occurred:", e)

    try:
        cc.connection.commit()
    finally:
        cursor.close()
    cc.connection.setautocommit(True)
